# Блокнот для подключения к Google Colab по ssh

Порядок действий
1. Пересохраните себе блокнот *Файл -> Сохранить копию на Диске*
2. При необходимости подключите аппаратный ускоритель *Среда выполнения -> Сменить среду выполнения -> Аппаратный ускоритель*
3. Запустите первую ячейку
4. Введите свой Ngrok-токен (получить на ngrok.com)
5. Задайте пароль для подключения к виртуальной машине
6. Выполните вторую ячейку установки Julia (опционально)
7. Подключитесь к виртуальной машине `ssh -p <port> root@<HostName>` 
8. После работы закройте ssh-туннель `kill $(pgrep ngrok)` 

In [ ]:
!pip install colab_ssh --upgrade

import getpass; 
from colab_ssh import launch_ssh; 

print('Ngrok-токен :'); 
authtoken = getpass.getpass();

print('Пароль :'); 
password = getpass.getpass();

launch_ssh(authtoken, password, False, 'eu');

Отредактируйте параметры установки
- `JULIA_VERSION` - версия Julia
- `JULIA_PACKAGES` - предустанавливаемые пакеты
- `JULIA_PACKAGES_IF_GPU` - пакеты для работы на ГПУ
- `GDRIVE` - подключение Диска Google

In [ ]:
%%shell 
set -e 
 
#---------------------------------------------#
JULIA_VERSION="1.6.1"
JULIA_PACKAGES="BenchmarkTools Revise"
JULIA_PACKAGES_IF_GPU="CUDA"
GDRIVE=false
#---------------------------------------------#
 
if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then 
  
  # Установка Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"` 
  echo "Установка Julia $JULIA_VERSION..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz
  tar xf /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz
 
  # Установка дополнительных пакетов
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do                        
    echo "Установка Julia-пакета $PKG..."
    julia -e 'using Pkg; Pkg.add("'"$PKG"'");'
  done 
 
  # Установка менеджера файлов Midnight Commander 
  echo "Установка mc"
  apt-get install mc
 
fi  
 
# Подключение Диска Google
if $GDRIVE ; then
  echo "Подключение Диска Google..."
  python3 -c "from google.colab import drive;  drive.mount('/content/drive')"
fi